In [26]:
from keras import optimizers
from keras.models import Sequential,Model
from keras.layers import Dense,CuDNNLSTM,Bidirectional
from keras.layers import Activation,Dropout,Flatten,GlobalMaxPool1D
import keras.optimizers as optimizer

import keras
import sys

In [2]:
temp = open('./Word-Level-Lyrics-Generation/Eminem.txt',encoding = 'UTF-8').read()

In [4]:
temp[0:250]

"spoken: \n Oh yeah, this is Eminem baby, back up in that motherfucking ass \n One time for your mother fucking mind, we represent the 313 \n You know what I'm saying?, 'cause they don't know shit about this \n For the 9-6 \n Ayo, my pen and paper cause a "

In [3]:
temp = temp.lower()
# $ =  Enter
temp = temp.replace('\n','$')

In [4]:
lyrics = []
for i,x in enumerate(temp):
  if x == '(':
    while(True):
      if temp[i].strip() != ')':
        i = i+1
      else:
        break
  elif x.isdigit() or (x not in['$','?','.',' ',' \ '] and not x.isalpha()):
    pass
  else:
    lyrics.append(x)

In [5]:
chars = sorted(list(set(lyrics)))

In [6]:
len(chars)

40

In [9]:
str(chars)

"[' ', '$', '.', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'à', 'á', 'â', 'ä', 'é', 'í', 'ñ', 'ó', 'ú', 'ü']"

In [7]:
intToChars = dict((i, c) for i, c in enumerate(chars))
charsToInt = dict((i, c) for c, i in enumerate(chars))

In [93]:
batch = 100
vocab_size = len(chars)
lyrics_size = len(lyrics)

In [94]:
X = []
y = []
for i in range(0,lyrics_size - batch - 1):
  tempX = lyrics[i:(i+batch)]
  tempY = lyrics[i+batch]
  
  X.append([charsToInt[ch] for ch in tempX])
  y.append(charsToInt[tempY])

(1725300, 100, 1)

In [95]:
import numpy as np
X = np.reshape(X,(len(y),batch,1))
y = np.reshape(y,(len(y),1))

In [96]:
X = X/40.0

In [97]:
labels = keras.utils.to_categorical(y, num_classes=40, dtype='float32')

In [98]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.10)

In [108]:
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
model = Sequential()

# add LSTM layer
model.add(Bidirectional(CuDNNLSTM(128, input_shape=(batch, 1),return_sequences=True)))
model.add(Bidirectional(CuDNNLSTM(128,return_sequences=True)))

model.add(Bidirectional(CuDNNLSTM(256,return_sequences=True)))
model.add(Bidirectional(CuDNNLSTM(256,return_sequences=True)))
model.add(Dropout(0.25))
model.add(GlobalMaxPool1D())
model.add(Dense(100, activation="elu"))
model.add(Dropout(0.4))
# add Softmax layer to output one character 
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
adam = optimizer.Adam(lr = 0.01)
model.compile(loss='categorical_crossentropy', metrics = ['accuracy'], optimizer='Adam')


In [ ]:
# train the model
adam = optimizer.Adam(lr = 0.01)
model.compile(loss='categorical_crossentropy', metrics = ['accuracy'], optimizer='Adam')

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1,
                              patience=1, min_lr=0.000000001)

filepath="weights-improvement-{acc:.2f}.h5"
lambdac = keras.callbacks.LambdaCallback(on_epoch_begin=generate)
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=3)
callbacks_list = [reduce_lr,checkpoint,lambdac]
model.fit(X_train, y_train, batch_size=128, epochs=50,validation_data = (X_test,y_test),verbose = 1,callbacks = callbacks_list)

Train on 1552770 samples, validate on 172530 samples
Epoch 1/50
  95488/1552770 [>.............................] - ETA: 22:36 - loss: 2.7701 - acc: 0.2383

In [19]:
def save():
    model.save_weights('my_model_weights.h5')
    model.save('my_model.h5')


In [110]:
def generate(epoch, logs):
    input_data = 'you can try and read my lyrics off of this paper before i lay em but you wont take the sting out the'
    for i in range(400):  
      temp = [charsToInt[ch] for ch in input_data]
      temp = np.asarray(temp)
      temp = temp.reshape((1,100,1))
      temp = temp/40.0 
      hum = model.predict(temp)
      hum = np.argmax(hum)
      ch = intToChars[hum]
      if ch != '$':
        sys.stdout.write(ch)
      else:
        print('')
      input_data = input_data[1:]
      input_data = input_data + ch

In [101]:
def generate1():
    input_data = 'to take a stand its '
    for i in range(600):  
      temp = [charsToInt[ch] for ch in input_data]
      temp = np.asarray(temp)
      temp = temp.reshape((1,100,1))
      temp = temp/40.0 
      hum = model.predict(temp)
      hum = np.argmax(hum)
      ch = intToChars[hum]
      if ch != '$':
        sys.stdout.write(ch)
      else:
        print('')
      input_data = input_data[1:]
      input_data = input_data + ch

In [83]:
generate()

the man who would jump in front of a minivan 
 since im still al inding the master 
 they say i love you baby 
  
 i dont mean to be the only one 
 they want me to move you more 
 you should be made me 
 i know you dont want it wit it 
 its murda murda munda 
 we aint nobody thinking that i was an armor 
 i dont must be the bad guy 
 i dont wanna but i gotta stay 
 whoa 
 is southe i constant is all over the same shit that i love you 
 and you cant stop me 
 i dont need you anymore 
 i still be mad at a bit 
 its the man they always keep a song and show your brain alone 
 and i dont mean to be